In [1]:
%load_ext autotime
import pandas as pd
import os
import geopandas as gpd
from glob import glob
from shapely.validation import make_valid
from tqdm.auto import tqdm
import time

In [2]:
files = pd.DataFrame({"filepath": glob("Asset Directory/**/*.shp", recursive=True)})
files["filename"] = files["filepath"].apply(lambda x: os.path.basename(x).replace(".shp", ""))
files["filesize_GB"] = files["filepath"].apply(lambda x: os.path.getsize(x) / 1e9)
files

,filepath,filename,filesize_GB
0,Asset Directory\DOC\Covenant_Areas.shp,Covenant_Areas,0.003984
1,Asset Directory\DOC\DOC_Campsites.shp,DOC_Campsites,0.000008
2,Asset Directory\DOC\DOC_Marine_Reserves.shp,DOC_Marine_Reserves,0.001981
3,Asset Directory\DOC\DOC_Offices.shp,DOC_Offices,0.000003
4,Asset Directory\DOC\DOC_Public_Conservation_La...,DOC_Public_Conservation_Land,0.139802
5,Asset Directory\DOC\DOC_Sanctuaries_to_Protect...,DOC_Sanctuaries_to_Protect_Marine_Mammals,0.003158
6,Asset Directory\DOC\Ecological_Districts.shp,Ecological_Districts,0.002513
7,Asset Directory\DOC\Internationally_Recognised...,Internationally_Recognised_Areas,0.004694
8,Asset Directory\Koordinate\new-zealand-2022-es...,new-zealand-2022-estimated-resident-population...,0.026755
9,Asset Directory\Koordinate\nz-cadastral-survey...,nz-cadastral-survey-network-marks,0.032303


In [3]:
df = gpd.read_file("Projections/JackettIsland_projection_output_polygon.shp")
df.geometry = df.geometry.apply(make_valid)
poly = df.unary_union

In [4]:
for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = gpd.clip(asset, poly)
  print(f"Time to clip: {time.time() - start}")
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("JackettIsland_affected_assets.gpkg", layer=row.filename, driver='GPKG')

  0%|          | 0/43 [00:00<?, ?it/s]

Now processing Covenant_Areas
Time to read file: 0.6575894355773926
Time to clip: 0.005000114440917969
Now processing DOC_Campsites


c:\Users\lalit\anaconda3\envs\environment\lib\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Time to read file: 0.15509033203125
Time to clip: 0.0040149688720703125
Now processing DOC_Marine_Reserves


c:\Users\lalit\anaconda3\envs\environment\lib\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


...

Time to read file: 22.920761346817017
Time to clip: 0.09838461875915527
Now processing nz-well-points-topo-150k
Time to read file: 0.10055232048034668


c:\Users\lalit\anaconda3\envs\environment\lib\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
c:\Users\lalit\anaconda3\envs\environment\lib\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Time to clip: 0.0
Now processing nz-windmill-points-topo-150k
Time to read file: 0.17473959922790527
Time to clip: 0.001999378204345703
Now processing lucas-nz-land-use-map-1990-2008-2012-2016-v011


c:\Users\lalit\anaconda3\envs\environment\lib\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Time to read file: 434.4261837005615
Time to clip: 1.6348981857299805


In [6]:
gpd.list_layers("JackettIsland_affected_assets.gpkg")

AttributeError: module 'geopandas' has no attribute 'list_layers'